# Mutation and concurrency

In [11]:
(def initial-board
      [[:- :k :-]
      [:- :- :-]
      [:- :K :-]])

#'user/initial-board

In [12]:
(defn board-map [f board]
  (vec (map #(vec (for [s %] (f s))) 
            board)))

#'user/board-map

In [13]:
(defn reset-board!
  "Resets the board state.  Generally these types of functions are a
   bad idea, but matters of page count force our hand."
  []
  (def board (board-map ref initial-board))
  (def to-move (ref [[:K [2 1]] [:k [0 1]]]))
  (def num-moves (ref 0)))

#'user/reset-board!

In [7]:
(defn neighbors
  ([size yx] (neighbors [[-1 0] [1 0] [0 -1] [0 1]]
                        size yx))
  ([deltas size yx]
     (filter (fn [new-yx]
                (every? #(< -1 % size) new-yx))
                (map #(vec (map + yx %))
                      deltas))))



#'user/neighbors

In [8]:
(def king-moves
  (partial neighbors 
           [[-1 -1] [-1 0] [-1 1] [0 -1] [0 1] [1 -1] [1 0] [1 1]] 3))

#'user/king-moves

In [14]:
(defn good-move?
  [to enemy-sq]
  (when (not= to enemy-sq) to))

#'user/good-move?

In [10]:
(defn choose-move
  "Randomly choose a legal move"
  [[[mover mpos] [_ enemy-pos]]]
  [mover (some #(good-move? % enemy-pos)
                (shuffle (king-moves mpos)))])

#'user/choose-move

In [21]:
(reset-board!)
(take 5 (repeatedly #(choose-move @to-move)))

([:K [1 0]] [:K [1 1]] [:K [1 2]] [:K [1 0]] [:K [1 0]])

In [23]:
(defn place [from to] to)

(defn move-piece [[piece dest] [[_ src] _]]
  (alter (get-in board dest) place piece)
   (alter (get-in board src ) place :-)
  (alter num-moves inc))

(defn update-to-move [move]
  (alter to-move #(vector (second %) move)))

(defn make-move []
  (let [move (choose-move @to-move)]
    (dosync (move-piece move @to-move))
    (dosync (update-to-move move))))

#'user/make-move

In [33]:
(make-move)

[[:K [1 1]] [:k [0 0]]]

In [35]:
; STM doesn't have locking mechanism but instead any concurrent transaction will check if it conflicts with other transaction before commit 
; Incase of conflict, it 'll retry

# Var

In [37]:
(defn print-read-eval []
  (println "*read-eval* is currently" *read-eval*))

#'user/print-read-eval

In [38]:
(defn binding-play []
  (print-read-eval)
  (binding [*read-eval* false]
    (print-read-eval))
  (print-read-eval))

#'user/binding-play

In [40]:
(binding-play)

*read-eval* is currently true
*read-eval* is currently false
*read-eval* is currently true


nil

In [56]:
; Create a far
(def ^:dynamic ngoc "dep trai")

#'user/ngoc

In [57]:
ngoc

"dep trai"

In [58]:
; to access the var instead of its value
#'ngoc

#'user/ngoc

In [62]:
(binding [ngoc "rat deptrai"]
    (prn ngoc))

"rat deptrai"


nil

In [65]:
; we can also use with-local-vars
(def x 42)
{:outer-var-value x
 :with-locals (with-local-vars [x 9]
                {:local-var x
                 :local-var-value (var-get x)})}

{:outer-var-value 42, :with-locals {:local-var #<Var: --unnamed-->, :local-var-value 9}}